<a href="https://colab.research.google.com/github/MsAsterisk1/ERM_GDRO/blob/main/civil_comments_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi -L  

GPU 0: NVIDIA GeForce RTX 2060 (UUID: GPU-100f20af-1b36-5adc-0795-f06b020c050b)


In [3]:
# !git clone https://github.com/MsAsterisk1/ERM_GDRO.git
# %cd /content/ERM_GDRO

In [4]:
# !git pull

In [5]:
# !pip install transformers

In [6]:
import pandas as pd
from utils.process_data_utils import  get_CivilComments_DataLoaders, get_CivilComments_Datasets, get_CivilComments_df
from loss import ERMLoss, GDROLoss
from train_eval import train, evaluate, train_epochs
import torch
from models import BertClassifier

Now lets try getting Bert working

In [7]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

In [8]:
df = get_CivilComments_df()

In [9]:
datasets = get_CivilComments_Datasets(CC_df=df, device=DEVICE)

In [10]:
tr,cv,test = get_CivilComments_DataLoaders(datasets=datasets)

This is how we would initialize model

In [11]:
model = BertClassifier(device=DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.01)
epochs = 5
loss_fn = ERMLoss(model, torch.nn.CrossEntropyLoss())
# loss_fn = GDROLoss(model, torch.nn.CrossEntropyLoss(), 0.01, 8, vector_subclass=True)

train_epochs(epochs, tr, cv, model, loss_fn, optimizer, vector_subclass=True, verbose=True, num_subclasses=18, save_weights_name='CC_ERM')


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

Epoch 1 / 5
Average training loss: 0.2264478290124004
For Cross Val:
Loss: 0.22557975351810455 Accuracy: 0.9049358123063302 
Accuracy over subgroups: [0.60699301 0.5849546  0.55865922 0.47916667 0.54296875 0.2
 0.67354597 0.62676056 0.68678029 0.92444444 0.93417969 0.83894449
 0.9668747  0.88861076 0.92592593 0.77479893 0.79950372 0.95365328] 
Worst Group Accuracy: 0.2
Epoch 2 / 5
Average training loss: 0.21623695677513527
For Cross Val:
Loss: 0.23032869398593903 Accuracy: 0.8903718459495352 
Accuracy over subgroups: [0.7020979  0.65888457 0.70949721 0.57291667 0.66992188 0.6
 0.79924953 0.77699531 0.72409028 0.88987654 0.90390625 0.70973612
 0.94959193 0.80913642 0.92592593 0.63538874 0.64813896 0.94201763] 
Worst Group Accuracy: 0.5729166666666666
Epoch 3 / 5
Average training loss: 0.2140838625643944
For Cross Val:
Loss: 0.20929887890815735 Accuracy: 0.9173085436033643 
Accuracy over subgroups: [0.4027972  0.40726329 0.34636872 0.26041667 0.33398438 0.
 0.37898687 0.39319249 0.503915

In [12]:
_ = evaluate(test, model, 18, vector_subclass=True, replacement=False, get_loss=False, verbose=True)

[autoreload of utils.process_data_utils failed: Traceback (most recent call last):
  File "C:\Users\tzeng1\AppData\Roaming\Python\Python310\site-packages\IPython\extensions\autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\tzeng1\AppData\Roaming\Python\Python310\site-packages\IPython\extensions\autoreload.py", line 455, in superreload
    module = reload(module)
  File "C:\Program Files\Python310\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "C:\Users\tzeng1\repos\ERM_GDRO\utils\process_data_utils.py", line 5, in <module>
    from wilds import get_dataset
ModuleNotFoundError: No module named 'wilds'
]


Accuracy: 0.9160200923891106 
Accuracy over subgroups: [0.52065365 0.50881057 0.53700658 0.4047619  0.43577136 0.21428571
 0.56668835 0.53294746 0.57015825 0.94748594 0.95711968 0.87133956
 0.97686142 0.92810458 0.98019802 0.85007496 0.88956841 0.97650703] 
Worst Group Accuracy: 0.21428571428571427


In [ ]:
model.load_state_dict(torch.load('./epoch_2_CC_ERM.wt'))
_ = evaluate(test, model, 18, vector_subclass=True, replacement=False, get_loss=False, verbose=True)

In [13]:
model.load_state_dict(torch.load('./epoch_3_CC_ERM.wt'))
_ = evaluate(test, model, 18, vector_subclass=True, replacement=False, get_loss=False, verbose=True)

Accuracy: 0.917021721905787 
Accuracy over subgroups: [0.43758511 0.42687225 0.40296053 0.32777778 0.34357714 0.14285714
 0.41314249 0.42742654 0.50293896 0.96543169 0.97355244 0.9211838
 0.98545575 0.95966387 0.99009901 0.92413793 0.93377599 0.98365992] 
Worst Group Accuracy: 0.14285714285714285


In [14]:
model = BertClassifier(device=DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.01)
epochs = 5
# loss_fn = ERMLoss(model, torch.nn.CrossEntropyLoss())
loss_fn = GDROLoss(model, torch.nn.CrossEntropyLoss(), 0.01, 18, vector_subclass=True)

train_epochs(epochs, tr, cv, model, loss_fn, optimizer, vector_subclass=True, verbose=True, num_subclasses=18, save_weights_name='CC_GDRO')


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

TypeError: train_epochs() missing 1 required positional argument: 'optimizer'

In [ ]:
_ = evaluate(test, model, 18, vector_subclass=True, replacement=False, get_loss=False, verbose=True)

In [ ]:
#run again to make sure 
_ = evaluate(test, model, 18, vector_subclass=True, replacement=False, get_loss=False, verbose=True)

In [ ]:
model.load_state_dict(torch.load('./epoch_3_CC_ERM.wt'))


In [ ]:
model.load_state_dict(torch.load('./epoch_2_CC_GDRO.wt'))
_ = evaluate(test, model, 18, vector_subclass=True, replacement=False, get_loss=False, verbose=True)